# K-Nearest Neighbour (KNN)
This document will look at implementing the KNN on a subset of the Titanic Dataset to classify individuals survived or perished in the disaster.

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing Data

In [2]:
raw_titanic_df = pd.read_csv(r'data\partial_titanic.csv')
print("Number of Observations: " + str(raw_titanic_df.iloc[:,1].count()))
raw_titanic_df.head(5)

Number of Observations: 891


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Selecting Features & Preprocessing
I will select some features that do not require a lot of engineering to be compatible with the model.
To Note:
- EDA not performed, undertaken an understanding of this dataset in other documents.
- Simple Selection to demonstrate the KNN Classifier.
- Pclass and Gender will be one-hot-encoded
- All features will be scaled

In [3]:
# Select a subset of features
sub_titanic_df = raw_titanic_df[["Pclass", "Sex", "Age", "Fare", "Survived"]].copy()
print(sub_titanic_df.isna().sum())
print("Length before NaN drop: " + str(sub_titanic_df.iloc[:, 1].count()))

Pclass        0
Sex           0
Age         177
Fare          0
Survived      0
dtype: int64
Length before NaN drop: 891


In [4]:
# Drop NaN's from Age column which is the only column they exist in
sub_titanic_df = sub_titanic_df[sub_titanic_df["Age"].notna()]
print("Length After NaN drop: " + str(sub_titanic_df.iloc[:, 1].count()))

Length After NaN drop: 714


In [5]:
# Split Target and Dependant Variables
X = sub_titanic_df.iloc[:, :-1]
y = sub_titanic_df.iloc[:, -1]

print("X Dependant Variables: ")
print(str(X.head()) + "\n")
print(r"y Target Predictor (Survived): ")
print(y.head())

X Dependant Variables: 
   Pclass     Sex   Age     Fare
0       3    male  22.0   7.2500
1       1  female  38.0  71.2833
2       3  female  26.0   7.9250
3       1  female  35.0  53.1000
4       3    male  35.0   8.0500

y Target Predictor (Survived): 
0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64


In [6]:
# Encoding categorical data
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
# 0 & 1 as thats the index of the columns to transform
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0,1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [7]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [8]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# Scale Train and Test sets by the Scaler that knows the Training Data
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
print("X_train Dependant Variables One-Hot-Encoded & Feature Scaled: ")
print(str(X_train[0:5]) + "\n")
print(r"y Target Predictor (Survived): ")
print(y_train[:5])

X_train Dependant Variables One-Hot-Encoded & Feature Scaled: 
[[-0.59822071 -0.55791981  1.00187091 -0.80068989  0.80068989 -1.75093891
  -0.08726366]
 [-0.59822071  1.79237229 -0.99813258 -0.80068989  0.80068989  0.87179144
  -0.40707716]
 [-0.59822071  1.79237229 -0.99813258  1.24892298 -1.24892298  0.0435608
  -0.40707716]
 [-0.59822071 -0.55791981  1.00187091  1.24892298 -1.24892298  0.733753
  -0.46876485]
 [ 1.67162383 -0.55791981 -0.99813258  1.24892298 -1.24892298  0.3886569
   0.27542498]]

y Target Predictor (Survived): 
850    0
149    0
747    1
40     0
383    1
Name: Survived, dtype: int64


# Building KNN Clasifier

In [10]:
# Training the K-NN model
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [11]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[96 15]
 [29 39]]


0.7541899441340782